In [1]:
from sklearn.model_selection import cross_val_score
>>> from sklearn.ensemble import RandomForestClassifier
>>> from sklearn.ensemble import ExtraTreesClassifier
>>> from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import permutation_importance
from sklearn import tree
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nurse_df = pd.read_csv('../output_data/model_data.csv')
nurse_df.head()

,Cum GPA,TEAS Count,MaxOfTestScore,failing grade count,Time to Test (days),Completion Time (days),PrecEducDesc,Grade,GradePass,HighSchool,Assoc,Bach,Masters
0,2.84,1.0,76.0,0,209.00,591.0,High School/GED/Some College,Pass,1.0,1.0,0.0,0.0,0.0
1,3.14,1.0,74.7,0,73.00,467.0,High School/GED/Some College,Pass,1.0,1.0,0.0,0.0,0.0
2,3.22,1.0,75.3,0,84.51,578.0,High School/GED/Some College,Pass,1.0,1.0,0.0,0.0,0.0
3,3.29,2.0,77.3,0,60.00,591.0,High School/GED/Some College,Fail,0.0,1.0,0.0,0.0,0.0
4,3.38,2.0,72.7,0,80.33,600.0,Associates,Pass,1.0,0.0,1.0,0.0,0.0


In [3]:
nurses_df = nurse_df.drop(['PrecEducDesc', 'Grade'], axis=1)
nurses_df.head()

,Cum GPA,TEAS Count,MaxOfTestScore,failing grade count,Time to Test (days),Completion Time (days),GradePass,HighSchool,Assoc,Bach,Masters
0,2.84,1.0,76.0,0,209.00,591.0,1.0,1.0,0.0,0.0,0.0
1,3.14,1.0,74.7,0,73.00,467.0,1.0,1.0,0.0,0.0,0.0
2,3.22,1.0,75.3,0,84.51,578.0,1.0,1.0,0.0,0.0,0.0
3,3.29,2.0,77.3,0,60.00,591.0,0.0,1.0,0.0,0.0,0.0
4,3.38,2.0,72.7,0,80.33,600.0,1.0,0.0,1.0,0.0,0.0


In [4]:
nurses_df_pass = nurses_df[nurses_df["GradePass"]== 1]
len(nurses_df_pass)

898

In [5]:
nurses_df_fail = nurses_df[nurses_df["GradePass"] != 1]
len(nurses_df_fail)

327

In [6]:
new_data = pd.concat([nurses_df_fail ,nurses_df_pass.sample(n=len(nurses_df_fail)) ],copy=True,ignore_index=True)
len(new_data)

654

In [7]:
target = new_data["GradePass"]
target_names = ["negative", "positive"]
target

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
649    1.0
650    1.0
651    1.0
652    1.0
653    1.0
Name: GradePass, Length: 654, dtype: float64

In [8]:
data = new_data.drop("GradePass", axis=1)
feature_names = data.columns
data.head()

,Cum GPA,TEAS Count,MaxOfTestScore,failing grade count,Time to Test (days),Completion Time (days),HighSchool,Assoc,Bach,Masters
0,3.29,2.0,77.3,0,60.00,591.0,1.0,0.0,0.0,0.0
1,2.78,2.0,74.0,0,369.00,465.0,1.0,0.0,0.0,0.0
2,2.79,3.0,74.0,0,116.00,591.0,1.0,0.0,0.0,0.0
3,3.27,2.0,68.7,1,124.58,578.0,0.0,1.0,0.0,0.0
4,2.76,2.0,73.3,0,158.00,460.0,1.0,0.0,0.0,0.0


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target)

In [29]:
rf = ExtraTreesClassifier(n_estimators=100, max_depth=4)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.774390243902439

In [11]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.4639092043931163, 'Cum GPA'),
 (0.12623721578236397, 'Completion Time (days)'),
 (0.11544246925427894, 'failing grade count'),
 (0.0947423574957452, 'MaxOfTestScore'),
 (0.07929088690570534, 'TEAS Count'),
 (0.07460641262480229, 'Time to Test (days)'),
 (0.013661535648090356, 'Bach'),
 (0.012427174764056258, 'Masters'),
 (0.01125415595553634, 'HighSchool'),
 (0.008428587176305095, 'Assoc')]

In [12]:
result = new_data.values
pd.DataFrame(result)

,0,1,2,3,4,5,6,7,8,9,10
0,3.29,2.0,77.3,0.0,60.00,591.0,0.0,1.0,0.0,0.0,0.0
1,2.78,2.0,74.0,0.0,369.00,465.0,0.0,1.0,0.0,0.0,0.0
2,2.79,3.0,74.0,0.0,116.00,591.0,0.0,1.0,0.0,0.0,0.0
3,3.27,2.0,68.7,1.0,124.58,578.0,0.0,0.0,1.0,0.0,0.0
4,2.76,2.0,73.3,0.0,158.00,460.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
649,3.02,4.0,76.0,0.0,153.35,599.0,1.0,0.0,1.0,0.0,0.0
650,3.09,4.0,73.3,0.0,77.00,593.0,1.0,1.0,0.0,0.0,0.0
651,3.09,2.0,66.0,0.0,173.00,467.0,1.0,1.0,0.0,0.0,0.0
652,2.88,1.0,68.7,0.0,116.34,599.0,1.0,1.0,0.0,0.0,0.0


In [18]:
y_predict = rf.predict(X_test)


array([0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1.,
       1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0.])

In [19]:
y_test

162    0.0
352    1.0
44     0.0
164    0.0
39     0.0
      ... 
248    0.0
314    0.0
590    1.0
429    1.0
200    0.0
Name: GradePass, Length: 164, dtype: float64

In [20]:
 # Calculate classification report
from sklearn.metrics import classification_report

print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

         0.0       0.82      0.73      0.78        83
         1.0       0.76      0.84      0.80        81

    accuracy                           0.79       164
   macro avg       0.79      0.79      0.79       164
weighted avg       0.79      0.79      0.79       164



In [21]:
class_report = classification_report(y_test, y_predict, output_dict=True)
class_report_df = pd.DataFrame(class_report).transpose()
class_report_df.to_csv('Outputs/extreme_trees_report.csv')
class_report_df

,precision,recall,f1-score,support
0.0,0.824324,0.734940,0.777070,83.000000
1.0,0.755556,0.839506,0.795322,81.000000
accuracy,0.786585,0.786585,0.786585,0.786585
macro avg,0.789940,0.787223,0.786196,164.000000
weighted avg,0.790359,0.786585,0.786085,164.000000
